In [14]:
%%writefile input.txt
|,39|,23|,20|,|,16|,14|
|7,_,_,_,21|16,_,_
|37,_,_,_,_,_,_
|16,_,_,_,_,_,|
|22,_,_,_,_,|,|
|13,_,_,|1,_,|,|
|16,_,_,|5,_,|,|

Overwriting input.txt


In [177]:
def form_sum_combination(k):
    values = [int(value)*(i+1) for i, value in enumerate("{:>09}".format(bin(k)[2:]))]
    values = set(values) - {0}
    values = sorted(values)
    # bool_values = [(i in values) for i in range(1, 10)]
    summation = sum(values)
    length = len(values)
    return length, summation, values

COMBINATIONS = {}
for k in range(1, 512):
    length, summation, values = form_sum_combination(k)
    COMBINATIONS.setdefault(length, {}).setdefault(summation, []).append(values)

COMBINATIONS_UNIQUE_VALUES = {}
for length, summations in combinations.items():
    for summation, value_combinations in summations.items():
        A = set()
        for value_combination in value_combinations:
            A = A.union(set(value_combination))    
        COMBINATIONS_UNIQUE_VALUES.setdefault(length, {}).setdefault(summation, A)

class ValueCell():
    def __init__(self, value, i, j):
        self.i = i
        self.j = j
        self.value = value
        self.parents = {}
        if value == "_":
            self.possible_values = set(range(1,10))
            self.definite_value = None
        else:
            self.possible_values = set()
            self.definite_value = int(value)
    
    def associate_parent(self, sumcell, row_or_col):
        sumcell.associated_cell_count[row_or_col] += 1
        sumcell.child_cells[row_or_col].append(self)
        self.parents[row_or_col] = sumcell
        
    def set_definite_value(self):
        assert len(self.possible_values) == 1
        self.definite_value = self.possible_values.pop()
        for row_or_col, parent in self.parents.items():
            for child_cell in parent.child_cells[row_or_col]:
                child_cell.possible_values = child_cell.possible_values - {self.definite_value}
            
        
class SumCell():
    def __init__(self, value, i, j):
        self.i = i
        self.j = j
        self.value = value
        a, b = value.split("|")
        self.sums = {"col": a if a == "" else int(a), "row": b if b == "" else int(b)}
        self.associated_cell_count = {"row": 0, "col": 0}
        self.child_cells = {"row": [], "col": []}
        self.possible_summations = {"row": None, "col": None}
        self.possible_values = {"row": None, "col": None}
        self.values_needed = True
    
    def set_values(self):
        if self.values_needed:
            for row_or_col in ["row", "col"]:
                length = self.associated_cell_count[row_or_col]
                if length:
                    summation = self.sums[row_or_col]
                    self.possible_values[row_or_col] = \
                        COMBINATIONS_UNIQUE_VALUES[length][summation]
                    self.possible_summations[row_or_col] = \
                        COMBINATIONS[length][summation]
            self.values_needed = False
    
    def update_possible_summations(self, value, row_or_col):
        self.possible_summations[row_or_col] = \
            [possible_summation for possible_summation in self.possible_summations[row_or_col]
             if value in possible_summation]

    def restrict_child_cells_by_sum(self):
        for row_or_col in ["row", "col"]:
            possible_values = self.possible_values[row_or_col]
            if possible_values is not None:
                associated_cell_count = self.associated_cell_count[row_or_col]
                if associated_cell_count:
                    for child_cell in self.child_cells[row_or_col]:
                        child_cell.possible_values = child_cell.possible_values.intersection(possible_values)
                    
class KakuroPuzzle():
    def __init__(self, filename):
        cells = []
        with open("input.txt", "r") as f:
            for i, row in enumerate(f):
                cells.append([])
                for j, cell in enumerate(row.strip().split(",")):
                    if "|" in cell:
                        cells[i].append(SumCell(cell, i, j))
                    else:
                        cells[i].append(ValueCell(cell, i, j))

        lengths = [len(row) for row in cells]
        assert min(lengths) == max(lengths), (min(lengths), max(lengths))
        self.m = len(cells)
        self.n = len(cells[0])
        self.shape = (m, n)
        
        rowSums = [None]*m
        columnSums = [None]*n
        for i, row in enumerate(cells):
            for j, cell in enumerate(row):
                if isinstance(cell, SumCell):
                    rowSums[i] = cell
                    columnSums[j] = cell
                else:
                    cell.associate_parent(rowSums[i], "row")
                    cell.associate_parent(columnSums[j], "col")
                    
        for i, row in enumerate(cells):
            for j, cell in enumerate(row):
                if isinstance(cell, SumCell):
                    cell.set_values()
                    cell.restrict_child_cells_by_sum()
                else:
                    pass
                
        self.cells = cells
                    
    def print_info(self):
        for i, row in enumerate(cells):
            for j, cell in enumerate(row):
                if isinstance(cell, SumCell):
                    print("row_children", [(child.i, child.j) for child in cell.child_cells["row"]])
                    print("col_children", [(child.i, child.j) for child in cell.child_cells["col"]])
                    print(cell.associated_cell_count)
                else:
                    print(cell.parents["row"].sums["row"], cell.parents["col"].sums["col"])     
                    
    def print_possible_values(self):
        printable_cells = []
        for i, row in enumerate(self.cells):
            printable_cells.append([])
            for j, cell in enumerate(row):
                if isinstance(cell, SumCell):
                    a, b = cell.value.split("|")
                    if a == "":
                        a = "__"
                    if b == "":
                        b = "__"
                    printable_cells[i].append("{:>9}|".format("{:>2}\{:>2}".format(a, b)))
                else:
                    printable_cells[i].append("{:>9}|".format("".join([str(x) for x in cell.possible_values])))
            printable_cells[i] = "".join(printable_cells[i])
        print("\n".join(printable_cells))

In [178]:
puzzle = KakuroPuzzle("input.txt")

In [179]:
puzzle.print_possible_values()

    __\__|    39\__|    23\__|    20\__|    __\__|    16\__|    14\__|
    __\ 7|        4|      124|      124|    21\16|       97|        9|
    __\37|   456789|  2345678| 23456789| 23456789| 23456789|     8956|
    __\16|       46|    12346|    12346|    12346|    12346|    __\__|
    __\22|   456789| 12345678|123456789|123456789|    __\__|    __\__|
    __\13|   456789|    45678|    __\ 1|        1|    __\__|    __\__|
    __\16|       97|        7|    __\ 5|        5|    __\__|    __\__|


In [129]:
for i, row in enumerate(cells):
    for j, cell in enumerate(row):
        if isinstance(cell, SumCell):
            pass
        else:
            if len(cell.possible_values) == 1:
                cell.set_definite_value()
                

In [130]:
for i, row in enumerate(cells):
    for j, cell in enumerate(row):
        if isinstance(cell, SumCell):
            print(i, j)
        else:
            if cell.definite_value is not None:
                print("Determined:", cell.definite_value)
            else:
                print(i, j, cell.possible_values)

0 0
0 1
0 2
0 3
0 4
0 5
0 6
1 0
Determined: 4
1 2 {1, 2}
1 3 {1, 2}
1 4
Determined: 7
Determined: 9
2 0
2 1 {8, 5, 7}
2 2 {2, 3, 4, 5, 6, 8}
2 3 {2, 3, 4, 5, 6, 7, 8, 9}
2 4 {2, 3, 4, 6, 7, 8, 9}
2 5 {2, 3, 4, 5, 6, 8, 9}
2 6 {8, 5, 6}
3 0
Determined: 6
3 2 {1, 2, 3, 4}
3 3 {1, 2, 3, 4}
3 4 {2, 3, 4}
3 5 {1, 2, 3, 4}
3 6
4 0
4 1 {8, 5, 7}
4 2 {1, 2, 3, 4, 5, 6, 8}
4 3 {1, 2, 3, 4, 5, 6, 7, 8, 9}
4 4 {2, 3, 4, 6, 7, 8, 9}
4 5
4 6
5 0
5 1 {8, 5, 7}
5 2 {8, 4, 5, 6}
5 3
Determined: 1
5 5
5 6
6 0
Determined: 9
Determined: 7
6 3
Determined: 5
6 5
6 6
